In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import base64
import addfips

Step 1: The first part of this process is pulling in the data that we gathered from different sources 

In [3]:
#import the Environmental data
with open('data/NTSIP_Public_Use_Data_2013-grouped-clean.csv') as f:
    df_ntsip_13 = pd.read_csv(f,delimiter=',') #import ntsip data 2013
with open('data/NTSIP_Public_Use_Data_2014-grouped-clean.csv') as f:
    df_ntsip_14 = pd.read_csv(f,delimiter=',') #import  ntsip data 2014
with open('data/air toxics - air concentration _ clean.csv') as f:
    df_air_tx_con = pd.read_csv(f,delimiter=',') #import air tox air concen.
with open('data/arsenic in community water system_clean.csv') as f:
    df_arsenic_wtr = pd.read_csv(f,delimiter=',') #import arsenic in community water
with open('data/power_plants.csv') as f:
    df_power_plants = pd.read_csv(f,delimiter=',') #import power plants data
with open('data/AMA_2017_2011_clean.csv') as f:
    df_ama = pd.read_csv(f,delimiter=',') #import ama data
with open('data/AQI_2011_Clean.csv') as f:
    df_aqi_11 = pd.read_csv(f,delimiter=',') #import aqi data 2011
with open('data/AQI_2018_Clean.csv') as f:
    df_aqi_18 = pd.read_csv(f,delimiter=',') #import aqi data 2018
with open('data/Average GHG emission per county fips - clean.csv') as f:
    df_epa_ghg = pd.read_csv(f,delimiter=',') #import epa emmissions data
with open('data/Echo_Data_Clean.csv') as f:
    df_echo = pd.read_csv(f,delimiter=',') #import echo data

In [4]:
#import the health data
with open('data/air toxics - percent cancer risk_clean.csv') as f:
    df_air_tx_cncr = pd.read_csv(f,delimiter=',') #import airtox percent cancer risk
with open('data/Number of Hospital Beds per county (1).csv') as f:
    df_hosp_beds = pd.read_csv(f,delimiter=',') #import hospital bed data
with open('data/SVI2018_US_COUNTY_clean.csv') as f:
    df_svi_18 = pd.read_csv(f,delimiter=',') #import SVI2018 (happiness index?)
with open('data/asthma prevalence in adults_clean_1192022.csv') as f:
    df_adlt_asthma = pd.read_csv(f,delimiter=',') #import adult asthma prevalence
with open('data/cancer prevalence-clean.csv') as f:
    df_cancer_prev = pd.read_csv(f,delimiter=',') #import adult asthma prevalence

Step 2: Goes through each dataset to prepare it for the merge by removing unnecessary columns and renaming columns to make the join successful. Each data frame should have a column for the fips entitled "CountyFIPS" along with the key variables for that dataset. If the column has a generic name it is renamed for idetification in the joined dataset
    

In [5]:
df_ntsip_13.head() #is this really relevent? what is the chemical exposure related to?

,CountyFIPS,Number of Chemical Exposure Incidents
0,6001,9
1,6005,1
2,6009,1
3,6011,2
4,6013,14


In [6]:
  df_air_tx_cncr.tail()


,StateFIPS,State,CountyFIPS,County,Year,Value in percent,Pollutant,Air Toxic - percent cancer risk
94285,56,Wyoming,56045,Weston,2011,5.20%,"Pollutant: 1,3-butadiene",0.052
94286,56,Wyoming,56045,Weston,2011,18.50%,"Pollutant: 1,3-butadiene",0.185
94287,56,Wyoming,56045,Weston,2011,3.40%,"Pollutant: 1,3-butadiene",0.034
94288,56,Wyoming,56045,Weston,2011,0%,"Pollutant: 1,3-butadiene",0.000
94289,56,Wyoming,56045,Weston,2011,72.80%,"Pollutant: 1,3-butadiene",0.728


In [9]:
  df_air_tx_cncr_11 = df_air_tx_cncr.drop(columns=['StateFIPS','State','County','Year'])
df_air_tx_cncr_11 = df_air_tx_cncr_11.rename({'Value in percent':'Air_toxin %'}, axis=1)
df_air_tx_cncr_11.head()

,CountyFIPS,Air_toxin %,Pollutant,Air Toxic - percent cancer risk
0,1001,0.70%,Pollutant: Benzene,0.007
1,1001,8%,Pollutant: Benzene,0.080
2,1001,63.40%,Pollutant: Benzene,0.634
3,1001,9%,Pollutant: Benzene,0.090
4,1001,0%,Pollutant: Benzene,0.000


In [10]:

  df_hosp_beds.head()
df_hosp_beds_16 =df_hosp_beds.drop(columns=['StateFIPS','State','County','Year'])
df_hosp_beds_16.head()


,CountyFIPS,Number of Beds
0,1001,50
1,1003,364
2,1005,47
3,1007,180
4,1009,25


In [12]:
  #df_air_tx_cncr.tail()
  #df_hosp_beds
df_svi_18=df_svi_18.drop(columns=['ST','ST_ABBR','STATE','LOCATION','COUNTY'])
df_svi_18 = df_svi_18.rename({'FIPS':'CountyFIPS'}, axis=1)
  df_svi_18.head()
   
   

,CountyFIPS,RPL_THEME1_socieconomic,RPL_THEME2_household_composition_and_disability,RPL_THEME3_minority_status_and_language,RPL_THEME4_Housing_type_and_transportation,RPL_THEMES_Social_Vulnerability_Index
0,35039,-999.0000,0.8758,0.9268,0.5409,-999.0000
1,1001,0.3631,0.5810,0.5947,0.3741,0.4354
2,1009,0.6143,0.3187,0.5915,0.1741,0.4242
3,1013,0.8455,0.9077,0.6447,0.6259,0.8653
4,1015,0.7866,0.8303,0.6507,0.6492,0.8252


In [13]:
df_adlt_asthma_18 = df_adlt_asthma.drop(columns=['StateFIPS','State','County','Year'])
df_adlt_asthma_18.head()
    #df_cancer_prev

,CountyFIPS,Value
0,1001,0.102
1,1003,0.096
2,1005,0.117
3,1007,0.103
4,1009,0.102


In [14]:
df_cancer_prev_18 = df_cancer_prev.drop(columns=['StateFIPS','State','County','Year'])
df_cancer_prev_18 = df_cancer_prev_18.rename({'Value':'%Cancer Prev'}, axis=1)
#df_adlt_asthma_18.head()
df_cancer_prev_18.head()

,CountyFIPS,%Cancer Prev,Percent Cancer Prevalence
0,1001,6.5%,0.065
1,1003,6.7%,0.067
2,1005,6.1%,0.061
3,1007,6.4%,0.064
4,1009,6.6%,0.066


In [9]:
df_air_tx_con.head()

,StateFIPS,State,CountyFIPS,County,Year,Value,Pollutant
0,1,Alabama,1001,Autauga,2011,0.54,Pollutant: Benzene
1,1,Alabama,1001,Autauga,2011,2.87,Pollutant: Formaldehyde
2,1,Alabama,1001,Autauga,2011,3.59,Pollutant: Acetaldehyde
3,1,Alabama,1001,Autauga,2011,0.55,Pollutant: Carbon tetrachloride
4,1,Alabama,1001,Autauga,2011,0.04,"Pollutant: 1,3-butadiene"


In [15]:
df_air_tx_con_11=df_air_tx_con.drop(columns=['StateFIPS','State','Year','County'])

df_air_tx_con_11.head()

 #df_air_tx_con.groupby("Pollutant")
    #for the pollutants (5) Benzene Fomaldehyde Acetaldehyde Carbon tetrachloride 1,2 butadiene, so there will always be at 
    #least 5 rows per county
#donziies!!


,CountyFIPS,Value,Pollutant
0,1001,0.54,Pollutant: Benzene
1,1001,2.87,Pollutant: Formaldehyde
2,1001,3.59,Pollutant: Acetaldehyde
3,1001,0.55,Pollutant: Carbon tetrachloride
4,1001,0.04,"Pollutant: 1,3-butadiene"


In [13]:
df_air_tx_con_11.shape #15 715/5  tells us the rows to expect and the counties we can assume are counted with in these observations

(15715, 3)

In [16]:
df_arsenic_wtr.head(15) #Arsenic in water
#df_arsenic_wtr = df_arsenic.dop([col])
#df_arsenic_wtr_grp= df_arsenic_wtr.groupby('CountyFIPS')
#df_arsenic_wtr_grp.head()

,StateFIPS,State,CountyFIPS,County,Year,Value,Maximum Contaminant Level,Maximum Contaminant Level.1
0,4,Arizona,4001,Apache,2018,3.60,Less than or Equal to MCL (2),Maximum Contaminant Level: Less than or Equal ...
1,4,Arizona,4001,Apache,2018,6.00,Less than or Equal to MCL (2),Maximum Contaminant Level: Less than or Equal ...
2,4,Arizona,4001,Apache,2018,1.00,Less than or Equal to MCL (2),Maximum Contaminant Level: Less than or Equal ...
3,4,Arizona,4001,Apache,2019,8.00,Less than or Equal to MCL (2),Maximum Contaminant Level: Less than or Equal ...
4,4,Arizona,4001,Apache,2019,0.50,Not Detected (3),Maximum Contaminant Level: Not Detected
5,4,Arizona,4001,Apache,2019,0.50,Not Detected (3),Maximum Contaminant Level: Not Detected
6,4,Arizona,4001,Apache,2019,2.90,Less than or Equal to MCL (2),Maximum Contaminant Level: Less than or Equal ...
7,4,Arizona,4003,Cochise,2018,5.25,Less than or Equal to MCL (2),Maximum Contaminant Level: Less than or Equal ...
8,4,Arizona,4003,Cochise,2018,1.70,Less than or Equal to MCL (2),Maximum Contaminant Level: Less than or Equal ...
9,4,Arizona,4003,Cochise,2018,1.70,Less than or Equal to MCL (2),Maximum Contaminant Level: Less than or Equal ...


In [17]:
#checking the columns with values
df_arsenic_wtr["Maximum Contaminant Level.1"].isna().value_counts() #interesting that a text field is showing int654 as dtype


False    10448
Name: Maximum Contaminant Level.1, dtype: int64

In [14]:
df_arsenic_wtr.shape #this is the data that determines the number of rows per county.

(10448, 8)

In [7]:
#clean the arsenic water data
df_arsenic_wtr=df_arsenic_wtr.drop(columns=['StateFIPS','State','County','Maximum Contaminant Level','Maximum Contaminant Level.1'])
df_arsenic_wtr.head()

,CountyFIPS,Year,Value
0,4001,2018,3.6
1,4001,2018,6.0
2,4001,2018,1.0
3,4001,2019,8.0
4,4001,2019,0.5


In [18]:
#drop fields for cleaner data
#df_arsenic_wtr.drop(columns=['Maximum Contaminant Level','Maximum Contaminant Level.1','Year', 'StateFIPS','State','County'])
df_arsenic_wtr_18 = df_arsenic_wtr.rename({'Value':'Arsenic_Avg'},axis=1) #changing the name of the value for the merge

In [19]:
df_arsenic_wtr_18 = df_arsenic_wtr_18.drop(columns=['Maximum Contaminant Level','Maximum Contaminant Level.1','Year', 'StateFIPS','State','County'])

In [20]:
#df_arsenic_wtr_18.head()
df_arsenic_wtr_18.groupby('CountyFIPS')
df_arsenic_wtr_18_mn = df_arsenic_wtr_18.groupby('CountyFIPS').mean()
df_arsenic_wtr_18_mn.head() #donzies!!

,Arsenic_Avg
CountyFIPS,
4001,3.214286
4003,4.004643
4005,3.551333
4007,2.581000
4009,3.606000


In [21]:
df_arsenic_wtr_18.head()

,CountyFIPS,Arsenic_Avg
0,4001,3.6
1,4001,6.0
2,4001,1.0
3,4001,8.0
4,4001,0.5


In [22]:
df_power_plants.head()

,Unnamed: 0,Plant sequence number,Data Year,Plant state abbreviation,Plant name,Plant transmission or distribution system owner name,Utility ID,Plant FIPS state code,Plant FIPS county code,Plant county name,Latitude,Longitude,Plant primary fuel,Plant primary coal/oil/gas/ other fossil fuel category
0,0,1,2019,AK,Sand Point,"TDX Sand Point Generating, LLC",63560,2,13.0,Aleutians East,55.339722,-160.497222,DFO,OIL
1,1,2,2019,AL,Barry,Alabama Power Co,195,1,97.0,Mobile,31.006900,-88.010300,BIT,COAL
2,2,3,2019,AL,Gadsden,Alabama Power Co,195,1,55.0,Etowah,34.012800,-85.970800,NG,GAS
3,3,4,2019,AL,Gorgas,Alabama Power Co,195,1,127.0,Walker,33.644344,-87.196486,BIT,COAL
4,4,5,2019,TX,Copper Station,El Paso Electric Co,5701,48,141.0,El Paso,31.756900,-106.375000,NG,GAS


In [23]:
#clean the power plant
#df_power_plants.head() #this data would be good for mapping but nothing else as it pertains to being merged with the final dataset
df_power_plants_19 = df_power_plants.drop(columns=['Unnamed: 0','Plant sequence number','Data Year','Plant state abbreviation','Plant name','Plant transmission or distribution system owner name','Utility ID','Plant FIPS state code','Plant county name'])
df_power_plants_19.head()


,Plant FIPS county code,Latitude,Longitude,Plant primary fuel,Plant primary coal/oil/gas/ other fossil fuel category
0,13.0,55.339722,-160.497222,DFO,OIL
1,97.0,31.006900,-88.010300,BIT,COAL
2,55.0,34.012800,-85.970800,NG,GAS
3,127.0,33.644344,-87.196486,BIT,COAL
4,141.0,31.756900,-106.375000,NG,GAS


In [24]:
df_power_plants_19 = df_power_plants_19.rename({'Plant FIPS county code':'CountyFIPS','Latitude':'PP_Lat','Longitude':'PP_Long'},axis=1) #changing the name of the values for the merge

In [25]:
df_power_plants_19=df_power_plants_19.drop(columns=['Plant primary coal/oil/gas/ other fossil fuel category'])
df_power_plants_19.head()

,CountyFIPS,PP_Lat,PP_Long,Plant primary fuel
0,13.0,55.339722,-160.497222,DFO
1,97.0,31.006900,-88.010300,BIT
2,55.0,34.012800,-85.970800,NG
3,127.0,33.644344,-87.196486,BIT
4,141.0,31.756900,-106.375000,NG


In [26]:
 df_aqi_11.head()

,Unnamed: 0,State,County,Year,Days with AQI,Hazardous Days,Max AQI,90th Percentile AQI,Days Ozone,cnty_state,county_id_fips
0,0,Alabama,Baldwin,2011,282,0,126,74,209,"Baldwin County, Alabama",1003.0
1,1,Alabama,Clay,2011,118,0,86,63,0,"Clay County, Alabama",1027.0
2,2,Alabama,Colbert,2011,279,0,101,61,197,"Colbert County, Alabama",1033.0
3,3,Alabama,DeKalb,2011,336,0,101,63,271,"DeKalb County, Alabama",1049.0
4,4,Alabama,Elmore,2011,242,0,105,64,242,"Elmore County, Alabama",1051.0


In [27]:
 df_aqi_11= df_aqi_11.drop(columns=['Unnamed: 0','State','County','Year','cnty_state'])
df_aqi_11.head()

,Days with AQI,Hazardous Days,Max AQI,90th Percentile AQI,Days Ozone,county_id_fips
0,282,0,126,74,209,1003.0
1,118,0,86,63,0,1027.0
2,279,0,101,61,197,1033.0
3,336,0,101,63,271,1049.0
4,242,0,105,64,242,1051.0


In [28]:
df_aqi_11_final = df_aqi_11.rename({'county_id_fips':'CountyFIPS'},axis=1) #changing the name of the values for the merge
df_aqi_11_final.head()

,Days with AQI,Hazardous Days,Max AQI,90th Percentile AQI,Days Ozone,CountyFIPS
0,282,0,126,74,209,1003.0
1,118,0,86,63,0,1027.0
2,279,0,101,61,197,1033.0
3,336,0,101,63,271,1049.0
4,242,0,105,64,242,1051.0


In [29]:
df_epa_ghg.head()

,county_id_fips,AVERAGE of GHG QUANTITY (METRIC TONS CO2e)
0,1001,1.325053e+06
1,1003,1.889378e+05
2,1005,3.635792e+04
3,1009,5.582886e+04
4,1015,4.340798e+04


In [30]:
df_epa_ghg= df_epa_ghg.rename({'county_id_fips':'CountyFIPS'},axis=1)
df_epa_ghg.head()

,CountyFIPS,AVERAGE of GHG QUANTITY (METRIC TONS CO2e)
0,1001,1.325053e+06
1,1003,1.889378e+05
2,1005,3.635792e+04
3,1009,5.582886e+04
4,1015,4.340798e+04


In [ ]:
## df_echo.head()

In [35]:
#df_echo_final = df_echo.drop(columns=['Unnamed: 0','FAC_STATE','FAC_COUNTY','FAC_FIPS_CODE','FAC_ZIP'])
#df_echo= df_echo.drop(columns=['FAC_ZIP'])
df_echo.head()

,AIR_FLAG,NPDES_FLAG,SDWIS_FLAG,RCRA_FLAG,TRI_FLAG,GHG_FLAG,CWA_QTRS_WITH_NC,CWA_COMPLIANCE_STATUS,CWA_SNC_FLAG,RCRA_QTRS_WITH_NC,RCRA_COMPLIANCE_STATUS,SDWA_COMPLIANCE_STATUS,SDWA_SNC_FLAG,FEC_TOTAL_PENALTIES,county_id_fips
0,N,N,Y,N,N,N,NaN,NaN,N,NaN,NaN,No Violation Identified,N,NaN,40109.0
1,N,N,Y,N,N,N,NaN,NaN,N,NaN,NaN,No Violation Identified,N,NaN,18113.0
2,N,N,Y,N,N,N,NaN,NaN,N,NaN,NaN,Inactive,N,NaN,NaN
3,N,N,N,Y,N,N,NaN,NaN,N,0.0,No Violation Identified,NaN,N,NaN,6031.0
4,Y,N,N,N,N,N,NaN,NaN,N,NaN,NaN,NaN,N,NaN,8017.0


In [36]:
df_echo_final =df_echo.rename({'county_id_fips':'CountyFIPS'}, axis=1)
df_echo_final.head()

,AIR_FLAG,NPDES_FLAG,SDWIS_FLAG,RCRA_FLAG,TRI_FLAG,GHG_FLAG,CWA_QTRS_WITH_NC,CWA_COMPLIANCE_STATUS,CWA_SNC_FLAG,RCRA_QTRS_WITH_NC,RCRA_COMPLIANCE_STATUS,SDWA_COMPLIANCE_STATUS,SDWA_SNC_FLAG,FEC_TOTAL_PENALTIES,CountyFIPS
0,N,N,Y,N,N,N,NaN,NaN,N,NaN,NaN,No Violation Identified,N,NaN,40109.0
1,N,N,Y,N,N,N,NaN,NaN,N,NaN,NaN,No Violation Identified,N,NaN,18113.0
2,N,N,Y,N,N,N,NaN,NaN,N,NaN,NaN,Inactive,N,NaN,NaN
3,N,N,N,Y,N,N,NaN,NaN,N,0.0,No Violation Identified,NaN,N,NaN,6031.0
4,Y,N,N,N,N,N,NaN,NaN,N,NaN,NaN,NaN,N,NaN,8017.0


In [37]:
#df_ntsip_13.head()
df_aqi_11_final.head()

,Days with AQI,Hazardous Days,Max AQI,90th Percentile AQI,Days Ozone,CountyFIPS
0,282,0,126,74,209,1003.0
1,118,0,86,63,0,1027.0
2,279,0,101,61,197,1033.0
3,336,0,101,63,271,1049.0
4,242,0,105,64,242,1051.0


Step 3: Create the combined datasets for project use

In [42]:
alldata =pd.concat([df_ntsip_13, df_air_tx_cncr_11, df_hosp_beds_16, df_svi_18, df_adlt_asthma_18, df_cancer_prev_18, df_air_tx_con_11, df_arsenic_wtr_18, df_power_plants_19, df_epa_ghg, df_echo_final, df_aqi_11_final], axis=0)
                  #df_epa_ghg,df_echo,df_air_tx_con_11,df_air_tx_cncr_11,df_hosp_beds_16, df_svi_18, df_adlt_asthma_18,df_cancer_prev_18,df_power_plants_19, df_arsenic_wtr_18, on=[CountyFIPS"])
alldata.head()

,CountyFIPS,Number of Chemical Exposure Incidents,Air_toxin %,Pollutant,Air Toxic - percent cancer risk,Number of Beds,RPL_THEME1_socieconomic,RPL_THEME2_household_composition_and_disability,RPL_THEME3_minority_status_and_language,RPL_THEME4_Housing_type_and_transportation,...,RCRA_QTRS_WITH_NC,RCRA_COMPLIANCE_STATUS,SDWA_COMPLIANCE_STATUS,SDWA_SNC_FLAG,FEC_TOTAL_PENALTIES,Days with AQI,Hazardous Days,Max AQI,90th Percentile AQI,Days Ozone
0,6001,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6005,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6009,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6011,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6013,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
alldata.shape #149 985..44 before and 146 843/36 after

(146843, 36)

In [43]:
#3 measures only GHG, arsenic, cancer prev
alldata_1= pd.concat([df_epa_ghg,df_arsenic_wtr_18_mn, df_cancer_prev_18], axis=0)
alldata_1.head()

,CountyFIPS,AVERAGE of GHG QUANTITY (METRIC TONS CO2e),Arsenic_Avg,%Cancer Prev,Percent Cancer Prevalence
0,1001,1.325053e+06,NaN,NaN,NaN
1,1003,1.889378e+05,NaN,NaN,NaN
2,1005,3.635792e+04,NaN,NaN,NaN
3,1009,5.582886e+04,NaN,NaN,NaN
4,1015,4.340798e+04,NaN,NaN,NaN


In [44]:
alldata_1.shape #6187/5

(6187, 5)

In [45]:
#2018 measures arsenic, cancer prev, asthma, svi
alldata_2018=pd.concat([df_svi_18,df_adlt_asthma_18,df_cancer_prev_18, df_arsenic_wtr_18], axis=0)
alldata_2018.head()

,CountyFIPS,RPL_THEME1_socieconomic,RPL_THEME2_household_composition_and_disability,RPL_THEME3_minority_status_and_language,RPL_THEME4_Housing_type_and_transportation,RPL_THEMES_Social_Vulnerability_Index,Value,%Cancer Prev,Percent Cancer Prevalence,Arsenic_Avg
0,35039,-999.0000,0.8758,0.9268,0.5409,-999.0000,NaN,NaN,NaN,NaN
1,1001,0.3631,0.5810,0.5947,0.3741,0.4354,NaN,NaN,NaN,NaN
2,1009,0.6143,0.3187,0.5915,0.1741,0.4242,NaN,NaN,NaN,NaN
3,1013,0.8455,0.9077,0.6447,0.6259,0.8653,NaN,NaN,NaN,NaN
4,1015,0.7866,0.8303,0.6507,0.6492,0.8252,NaN,NaN,NaN,NaN


In [46]:
alldata_2018.shape #19 874/10

(19874, 10)

Step 4:Export all the files

In [47]:
import csv #export the file name to be used
file1_name="All_Team_61_Data.csv"
#file2_name="Small_Team_61_Data.csv"
#file3_name ="All_2018_Team_61_Data.csv"

alldata.to_csv(file1_name)
#alldata_1.to_csv(file2_name)
#alldata_2018.to_csv(file3_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [48]:
import csv #export the file name to be used
#file1_name="All_Team_61_Data.csv"
file2_name="Small_Team_61_Data.csv"
#file3_name ="All_2018_Team_61_Data.csv"

#alldata.to_csv(file1_name)
alldata_1.to_csv(file2_name)
#alldata_2018.to_csv(file3_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [49]:
import csv #export the file name to be used
#file1_name="All_Team_61_Data.csv"
#file2_name="Small_Team_61_Data.csv"
file3_name ="All_2018_Team_61_Data.csv"

#alldata.to_csv(file1_name)
#alldata_1.to_csv(file2_name)
alldata_2018.to_csv(file3_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
